In [408]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, log_loss
import operator
import json
from IPython import display
import os
import warnings

np.random.seed(0)
warnings.filterwarnings("ignore")
THRESHOLD = 4


In [409]:
for dirname, _, filenames in os.walk('/kaggle/input'):

    for filename in filenames:

        print(os.path.join(dirname, filename))

/kaggle/input/padhai-mp-neuron-like-unlike-classification/sample_submission.csv
/kaggle/input/padhai-mp-neuron-like-unlike-classification/test.csv
/kaggle/input/padhai-mp-neuron-like-unlike-classification/train.csv


Task: To predict whether the user likes the mobile phone or not. <br>
Assumption: If the average rating of mobile >= threshold, then the user likes it, otherwise not.

<b>Missing values:</b><br>
'Also Known As'(459),'Applications'(421),'Audio Features'(437),'Bezel-less display'(266),'Browser'(449),'Build Material'(338),'Co-Processor'(451),'Display Colour'(457),'Mobile High-Definition Link(MHL)'(472),'Music'(447)
'Email','Fingerprint Sensor Position'(174),'Games'(446),'HDMI'(454),'Heart Rate Monitor'(467),'IRIS Scanner'(467),
'Optical Image Stabilisation'(219),'Other Facilities'(444),'Phone Book'(444),'Physical Aperture'(87),'Quick Charging'(122),'Ring Tone'(444),'Ruggedness'(430),SAR Value(315),'SIM 3'(472),'SMS'(470)', 'Screen Protection'(229),'Screen to Body Ratio (claimed by the brand)'(428),'Sensor'(242),'Software Based Aperture'(473),
'Special Features'(459),'Standby time'(334),'Stylus'(473),'TalkTime'(259), 'USB Type-C'(374),'Video Player'(456),
'Video Recording Features'(458),'Waterproof'(398),'Wireless Charging','USB OTG Support'(159), 'Video ,'Recording'(113),'Java'(471),'Browser'(448)

<b>Very low variance:</b><br>
'Architecture'(most entries are 64-bit),'Audio Jack','GPS','Loudspeaker','Network','Network Support','Other Sensors'(28),'SIM Size', 'VoLTE'


<b>Multivalued:</b><br>
'Colours','Custom UI','Model'(1),'Other Sensors','Launch Date'

<b>Not important:</b><br>
'Bluetooth', 'Settings'(75),'Wi-Fi','Wi-Fi Features'

<b>Doubtful:</b><br>
'Aspect Ratio','Autofocus','Brand','Camera Features','Fingerprint Sensor'(very few entries are missing),
'Fingerprint Sensor Position', 'Graphics'(multivalued),'Image resolution'(multivalued),'SIM Size','Sim Slot(s)', 'User Available Storage', 'SIM 1', 'SIM 2','Shooting Modes', 'Touch Screen'(24), 'USB Connectivity'
    
<b>To check:</b><br>
'Display Type','Expandable Memory','FM Radio'

<b>High Correlation with other features</b><br>
'SIM Slot(s)' high correlation with SIM1
'Weight' has high high correlation with capacity , screen-to-body ratio
'Height' - screen size is also there
    
<b>Given a mobile, we can't directly get these features</b><br>
'Rating Count', 'Review Count'

<b>Keeping:</b><br>
'Capacity','Flash'(17),'Height'(22),'Internal Memory'(20, require cleaning),'Operating System'(25, require cleaning), 'Pixel Density'(1, clean it),'Processor'(22, clean it), 'RAM'(17, clean), 'Rating','Resolution'(cleaning), 'Screen Resolution','Screen Size', 'Thickness'(22), 'Type','User Replaceable','Weight'(cleaning),'Sim Size'(), 'Other Sensors'(28), 'Screen to Body Ratio (calculated)','Width',


In [410]:
# read data from file
train = pd.read_csv("../input/padhai-mp-neuron-like-unlike-classification/train.csv") 
test = pd.read_csv("../input/padhai-mp-neuron-like-unlike-classification/test.csv")

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 355
Number of features in train: 99
Number of data points in test: 119
Number of features in test: 98


In [411]:
def data_clean(data):
    
    # Let's first remove all missing value features
    columns_to_remove = ['Also Known As','Applications','Audio Features','Bezel-less display'
                         'Browser','Build Material','Co-Processor','Browser'
                         'Display Colour','Mobile High-Definition Link(MHL)',
                         'Music', 'Email','Fingerprint Sensor Position',
                         'Games','HDMI','Heart Rate Monitor','IRIS Scanner', 
                         'Optical Image Stabilisation','Other Facilities',
                         'Phone Book','Physical Aperture','Quick Charging',
                         'Ring Tone','Ruggedness','SAR Value','SIM 3','SMS',
                         'Screen Protection','Screen to Body Ratio (claimed by the brand)',
                         'Sensor','Software Based Aperture', 'Special Features',
                         'Standby time','Stylus','TalkTime', 'USB Type-C',
                         'Video Player', 'Video Recording Features','Waterproof',
                         'Wireless Charging','USB OTG Support', 'Video Recording','Java']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    #Features having very low variance 
    columns_to_remove = ['Architecture','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    # Multivalued:
    columns_to_remove = ['Architecture','Launch Date','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE', 'Custom UI']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    # Not much important
    columns_to_remove = ['Bluetooth', 'Settings','Wi-Fi','Wi-Fi Features']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]
    
    return data

# Removing features

In [412]:
train = data_clean(train)
test = data_clean(test)

removing all those data points in which more than 15 features are missing 

In [413]:
train = train[(train.isnull().sum(axis=1) <= 15)]
# You shouldn't remove data points from test set
#test = test[(test.isnull().sum(axis=1) <= 15)]

In [414]:
# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 47
Number of data points in test: 119
Number of features in test: 46


# Filling Missing values

In [415]:
def for_integer(test):
    try:
        test = test.strip()
        return int(test.split(' ')[0])
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass

def for_string(test):
    try:
        test = test.strip()
        return (test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

def for_float(test):
    try:
        test = test.strip()
        return float(test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

    
def for_Internal_Memory(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[1] == 'GB':
            return int(test[0])
        if test[1] == 'MB':
#             print("here")
            return (int(test[0]) * 0.001)
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass
    
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass


In [416]:
def data_clean_2(x):
    data = x.copy()
    
    data['Capacity'] = data['Capacity'].apply(for_integer)

    data['Height'] = data['Height'].apply(for_float)
    data['Height'] = data['Height'].fillna(data['Height'].mean())

    data['Internal Memory'] = data['Internal Memory'].apply(for_Internal_Memory)

    data['Pixel Density'] = data['Pixel Density'].apply(for_integer)

    data['Internal Memory'] = data['Internal Memory'].fillna(data['Internal Memory'].median())
    data['Internal Memory'] = data['Internal Memory'].astype(int)

    data['RAM'] = data['RAM'].apply(for_integer)
    data['RAM'] = data['RAM'].fillna(data['RAM'].median())
    data['RAM'] = data['RAM'].astype(int)

    data['Resolution'] = data['Resolution'].apply(for_integer)
    data['Resolution'] = data['Resolution'].fillna(data['Resolution'].median())
    data['Resolution'] = data['Resolution'].astype(int)

    data['Screen Size'] = data['Screen Size'].apply(for_float)

    data['Thickness'] = data['Thickness'].apply(for_float)
    data['Thickness'] = data['Thickness'].fillna(data['Thickness'].mean())
    data['Thickness'] = data['Thickness'].round(2)

    data['Type'] = data['Type'].fillna('Li-Polymer')

    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].apply(for_float)
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].fillna(data['Screen to Body Ratio (calculated)'].mean())
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].round(2)

    data['Width'] = data['Width'].apply(for_float)
    data['Width'] = data['Width'].fillna(data['Width'].mean())
    data['Width'] = data['Width'].round(2)

    data['Flash'][data['Flash'].isna() == True] = "Other"

    data['User Replaceable'][data['User Replaceable'].isna() == True] = "Other"

    data['Num_cores'] = data['Processor'].apply(for_string)
    data['Num_cores'][data['Num_cores'].isna() == True] = "Other"


    data['Processor_frequency'] = data['Processor'].apply(find_freq)
    #because there is one entry with 208MHz values, to convert it to GHz
    data['Processor_frequency'][data['Processor_frequency'] > 200] = 0.208
    data['Processor_frequency'] = data['Processor_frequency'].fillna(data['Processor_frequency'].mean())
    data['Processor_frequency'] = data['Processor_frequency'].round(2)

    data['Camera Features'][data['Camera Features'].isna() == True] = "Other"

    #simplifyig Operating System to os_name for simplicity
    data['os_name'] = data['Operating System'].apply(for_string)
    data['os_name'][data['os_name'].isna() == True] = "Other"

    data['Sim1'] = data['SIM 1'].apply(for_string)

    data['SIM Size'][data['SIM Size'].isna() == True] = "Other"

    data['Image Resolution'][data['Image Resolution'].isna() == True] = "Other"

    data['Fingerprint Sensor'][data['Fingerprint Sensor'].isna() == True] = "Other"

    data['Expandable Memory'][data['Expandable Memory'].isna() == True] = "No"

    data['Weight'] = data['Weight'].apply(for_integer)
    data['Weight'] = data['Weight'].fillna(data['Weight'].mean())
    data['Weight'] = data['Weight'].astype(int)

    data['SIM 2'] = data['SIM 2'].apply(for_string)
    data['SIM 2'][data['SIM 2'].isna() == True] = "Other"
    
    return data

In [417]:
train = data_clean_2(train)
test = data_clean_2(test)

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 51
Number of data points in test: 119
Number of features in test: 50


Not very important feature

In [418]:
def data_clean_3(x):
    
    data = x.copy()

    columns_to_remove = ['User Available Storage','SIM Size','Chipset','Processor','Autofocus','Aspect Ratio','Touch Screen',
                        'Bezel-less display','Operating System','SIM 1','USB Connectivity','Other Sensors','Graphics','FM Radio',
                        'NFC','Shooting Modes','Browser','Display Colour' ]

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = [ 'Screen Resolution','User Replaceable','Camera Features',
                        'Thickness', 'Display Type']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = ['Fingerprint Sensor', 'Flash', 'Rating Count', 'Review Count','Image Resolution','Type','Expandable Memory',\
                        'Colours','Width','Model']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    return data

In [419]:
train = data_clean_3(train)
test = data_clean_3(test)

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 18
Number of data points in test: 119
Number of features in test: 17


In [420]:
# one hot encoding

train_ids = train['PhoneId']
test_ids = test['PhoneId']

cols = list(test.columns)
cols.remove('PhoneId')
cols.insert(0, 'PhoneId')

combined = pd.concat([train.drop('Rating', axis=1)[cols], test[cols]])
print(combined.shape)
print(combined.columns)

combined = pd.get_dummies(combined)
print(combined.shape)
print(combined.columns)

train_new = combined[combined['PhoneId'].isin(train_ids)]
test_new = combined[combined['PhoneId'].isin(test_ids)]

(460, 17)
Index(['PhoneId', 'Height', 'Screen to Body Ratio (calculated)', 'Capacity',
       'Resolution', 'Internal Memory', 'Processor_frequency', 'Sim1',
       'Screen Size', 'RAM', 'SIM 2', 'Pixel Density', 'Num_cores', 'os_name',
       'SIM Slot(s)', 'Brand', 'Weight'],
      dtype='object')
(460, 87)
Index(['PhoneId', 'Height', 'Screen to Body Ratio (calculated)', 'Capacity',
       'Resolution', 'Internal Memory', 'Processor_frequency', 'Screen Size',
       'RAM', 'Pixel Density', 'Weight', 'Sim1_2G', 'Sim1_3G', 'Sim1_4G',
       'SIM 2_2G', 'SIM 2_3G', 'SIM 2_4G', 'SIM 2_Other', 'Num_cores_312',
       'Num_cores_Deca', 'Num_cores_Dual', 'Num_cores_Hexa', 'Num_cores_Octa',
       'Num_cores_Other', 'Num_cores_Quad', 'Num_cores_Tru-Octa',
       'os_name_Android', 'os_name_Blackberry', 'os_name_KAI', 'os_name_Nokia',
       'os_name_Other', 'os_name_Tizen', 'os_name_iOS',
       'SIM Slot(s)_Dual SIM, GSM+CDMA', 'SIM Slot(s)_Dual SIM, GSM+GSM',
       'SIM Slot(s)_Dual SIM, 

In [421]:
train_new = train_new.merge(train[['PhoneId', 'Rating']], on='PhoneId')

In [422]:
# check the number of features and data points in train
print("Number of data points in train: %d" % train_new.shape[0])
print("Number of features in train: %d" % train_new.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test_new.shape[0])
print("Number of features in test: %d" % test_new.shape[1])

Number of data points in train: 341
Number of features in train: 88
Number of data points in test: 119
Number of features in test: 87


In [423]:
train_new.head()

,PhoneId,Height,Screen to Body Ratio (calculated),Capacity,Resolution,Internal Memory,Processor_frequency,Screen Size,RAM,Pixel Density,...,Brand_Spice,Brand_Tecno,Brand_Ulefone,Brand_VOTO,Brand_Vivo,Brand_Xiaomi,Brand_Xiaomi Poco,Brand_Yu,Brand_iVooMi,Rating
0,0,157.9,80.68,4000,20,64,1.8,6.26,4,403,...,0,0,0,0,0,1,0,0,0,4.5
1,1,156.2,80.85,4230,8,32,1.8,6.20,3,271,...,0,0,0,0,0,0,0,0,0,4.5
2,2,157.0,83.68,3500,25,32,2.1,6.30,3,409,...,0,0,0,0,0,0,0,0,0,4.4
3,4,159.8,74.78,3300,24,64,2.2,6.00,4,411,...,0,0,0,0,0,0,0,0,0,4.3
4,5,160.4,84.23,3750,16,64,2.2,6.50,4,396,...,0,0,0,0,0,0,0,0,0,4.4


In [424]:
test_new.head()

,PhoneId,Height,Screen to Body Ratio (calculated),Capacity,Resolution,Internal Memory,Processor_frequency,Screen Size,RAM,Pixel Density,...,Brand_Sony,Brand_Spice,Brand_Tecno,Brand_Ulefone,Brand_VOTO,Brand_Vivo,Brand_Xiaomi,Brand_Xiaomi Poco,Brand_Yu,Brand_iVooMi
0,3,156.2,80.85,4230,5,16,1.8,6.20,2,271,...,0,0,0,0,0,0,0,0,0,0
1,11,156.0,81.60,5000,12,64,1.8,6.20,4,402,...,0,0,0,0,0,0,0,0,0,0
2,13,156.7,83.84,3500,25,64,2.0,6.30,6,409,...,0,0,0,0,0,0,0,0,0,0
3,16,156.7,83.84,3500,16,64,2.0,6.30,4,409,...,0,0,0,0,0,0,0,0,0,0
4,19,158.6,77.43,4000,20,64,1.8,5.99,4,403,...,0,0,0,0,0,0,1,0,0,0


## Dummy Solution

In [425]:
"""
WRITE YOUR MODELLING CODE HERE
"""
df1=train_new
df1.groupby('Rating').count()

,PhoneId,Height,Screen to Body Ratio (calculated),Capacity,Resolution,Internal Memory,Processor_frequency,Screen Size,RAM,Pixel Density,...,Brand_Sony,Brand_Spice,Brand_Tecno,Brand_Ulefone,Brand_VOTO,Brand_Vivo,Brand_Xiaomi,Brand_Xiaomi Poco,Brand_Yu,Brand_iVooMi
Rating,,,,,,,,,,,,,,,,,,,,,
2.5,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2.8,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
3.0,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
3.1,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
3.2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3.3,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
3.4,11,11,11,11,11,11,11,11,11,11,...,11,11,11,11,11,11,11,11,11,11
3.5,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
3.6,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8


In [426]:
from sklearn.model_selection import train_test_split

mobile_X_train=train_new.drop(["Rating","PhoneId"], axis=1)
x=mobile_X.values
stdr=MinMaxScaler()
x_std=stdr.fit_transform(x)
mobile_X_train_std=pd.DataFrame(x_std)

In [427]:
mobile_X_train_std.head()

,0,1,2,3,4,5,6,7,8,9,...,76,77,78,79,80,81,82,83,84,85
0,0.723077,0.891040,0.248748,0.782609,0.1250,0.444444,0.921241,0.005871,0.600462,0.455882,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.694017,0.893693,0.267947,0.260870,0.0625,0.444444,0.906921,0.003914,0.295612,0.404412,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.707692,0.937871,0.207012,1.000000,0.0625,0.611111,0.930788,0.003914,0.614319,0.404412,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.755556,0.798938,0.190317,0.956522,0.1250,0.666667,0.859189,0.005871,0.618938,0.408088,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.765812,0.946456,0.227880,0.608696,0.1250,0.666667,0.978520,0.005871,0.584296,0.430147,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [428]:
mobile_X_train.shape[0]

341

In [429]:
mobile_X_train_IDS=train_new['PhoneId']

In [430]:
mobile_Y_train=train_new['Rating']

In [431]:
print(mobile_Y_train.values)

[4.5 4.5 4.4 4.3 4.4 4.5 4.3 4.1 4.3 4.5 4.5 4.  4.4 4.4 4.4 4.4 4.5 4.4
 4.4 4.4 4.4 4.5 4.4 4.3 4.2 4.3 4.3 4.6 4.4 4.5 3.9 4.2 4.3 4.2 4.3 4.3
 4.2 4.4 4.1 3.8 4.4 4.6 4.3 4.5 4.5 4.2 3.9 4.3 4.  4.3 4.3 3.9 4.2 4.5
 4.  4.6 4.2 2.8 4.4 4.3 4.2 4.4 4.4 4.  4.4 4.4 4.1 4.5 4.3 3.9 4.3 4.1
 4.1 4.2 3.4 4.2 4.6 3.8 4.3 4.5 4.2 4.5 4.5 4.  4.1 4.4 4.5 4.1 4.5 4.3
 4.5 3.9 3.9 4.4 4.4 4.4 4.  4.1 4.1 4.4 4.3 4.3 4.2 4.3 3.4 4.3 4.3 4.2
 4.6 4.8 4.4 3.4 3.7 4.2 4.3 4.4 4.3 4.2 4.1 4.3 4.  3.9 4.4 4.4 3.9 3.1
 3.7 4.2 4.4 4.  4.  4.1 4.2 4.3 4.2 4.3 4.1 3.6 4.1 4.  3.8 3.7 4.3 4.1
 4.1 4.5 4.  4.3 3.6 4.1 4.5 3.5 4.  4.3 4.6 3.5 3.9 4.1 3.9 3.8 4.4 4.3
 4.  3.9 4.4 4.7 3.9 4.5 3.7 4.  4.3 4.1 4.8 4.1 4.  4.7 4.  4.1 4.  3.4
 4.1 3.9 3.1 4.1 3.7 3.4 3.9 4.3 4.1 3.7 3.8 4.  3.8 4.6 3.4 3.3 3.2 3.5
 4.3 4.1 4.  3.8 4.3 4.1 4.  3.8 3.8 3.9 4.1 3.7 4.2 4.  4.2 4.4 4.1 3.4
 4.2 4.4 3.9 3.9 4.4 4.2 4.5 4.5 4.2 4.  4.2 3.5 3.7 4.3 5.  3.5 4.4 3.7
 4.3 3.9 4.3 3.8 3.8 4.3 5.  4.1 3.  4.4 3.  4.  4.

In [432]:
mobile_Y_train.shape[0]

341

In [433]:
test_new.head()

,PhoneId,Height,Screen to Body Ratio (calculated),Capacity,Resolution,Internal Memory,Processor_frequency,Screen Size,RAM,Pixel Density,...,Brand_Sony,Brand_Spice,Brand_Tecno,Brand_Ulefone,Brand_VOTO,Brand_Vivo,Brand_Xiaomi,Brand_Xiaomi Poco,Brand_Yu,Brand_iVooMi
0,3,156.2,80.85,4230,5,16,1.8,6.20,2,271,...,0,0,0,0,0,0,0,0,0,0
1,11,156.0,81.60,5000,12,64,1.8,6.20,4,402,...,0,0,0,0,0,0,0,0,0,0
2,13,156.7,83.84,3500,25,64,2.0,6.30,6,409,...,0,0,0,0,0,0,0,0,0,0
3,16,156.7,83.84,3500,16,64,2.0,6.30,4,409,...,0,0,0,0,0,0,0,0,0,0
4,19,158.6,77.43,4000,20,64,1.8,5.99,4,403,...,0,0,0,0,0,0,1,0,0,0


In [434]:
mobile_X_test=test_new.drop(['PhoneId'], axis=1)

In [435]:
x=mobile_X_test.values
stdr=MinMaxScaler()
x_std=stdr.fit_transform(x)
mobile_X_test_std=pd.DataFrame(x_std)

In [436]:
mobile_X_test_std.head()

,0,1,2,3,4,5,6,7,8,9,...,76,77,78,79,80,81,82,83,84,85
0,0.717496,0.931169,0.806533,0.130435,0.0625,0.376344,0.936170,0.001957,0.248811,0.557522,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.714286,0.942441,1.000000,0.434783,0.2500,0.376344,0.936170,0.005871,0.456418,0.884956,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.725522,0.976105,0.623116,1.000000,0.2500,0.483871,0.957447,0.009785,0.467512,0.566372,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.725522,0.976105,0.623116,0.608696,0.2500,0.483871,0.957447,0.005871,0.467512,0.566372,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.756019,0.879772,0.748744,0.782609,0.2500,0.376344,0.891489,0.005871,0.458003,0.672566,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [437]:
mobile_X_test_IDs=test_new['PhoneId']

In [438]:
mobile_Y_train.head()

0    4.5
1    4.5
2    4.4
3    4.3
4    4.4
Name: Rating, dtype: float64

In [439]:
mobile_Y_train_binarised=mobile_Y_train.map(lambda x:1 if x>=4 else 0)

In [440]:
mobile_Y_train_binarised.head()

0    1
1    1
2    1
3    1
4    1
Name: Rating, dtype: int64

In [441]:
for i in range(mobile_Y_train.shape[0]):
    print(mobile_Y_train[i], ' ', mobile_Y_train_binarised[i])

4.5   1
4.5   1
4.4   1
4.3   1
4.4   1
4.5   1
4.3   1
4.1   1
4.3   1
4.5   1
4.5   1
4.0   1
4.4   1
4.4   1
4.4   1
4.4   1
4.5   1
4.4   1
4.4   1
4.4   1
4.4   1
4.5   1
4.4   1
4.3   1
4.2   1
4.3   1
4.3   1
4.6   1
4.4   1
4.5   1
3.9   0
4.2   1
4.3   1
4.2   1
4.3   1
4.3   1
4.2   1
4.4   1
4.1   1
3.8   0
4.4   1
4.6   1
4.3   1
4.5   1
4.5   1
4.2   1
3.9   0
4.3   1
4.0   1
4.3   1
4.3   1
3.9   0
4.2   1
4.5   1
4.0   1
4.6   1
4.2   1
2.8   0
4.4   1
4.3   1
4.2   1
4.4   1
4.4   1
4.0   1
4.4   1
4.4   1
4.1   1
4.5   1
4.3   1
3.9   0
4.3   1
4.1   1
4.1   1
4.2   1
3.4   0
4.2   1
4.6   1
3.8   0
4.3   1
4.5   1
4.2   1
4.5   1
4.5   1
4.0   1
4.1   1
4.4   1
4.5   1
4.1   1
4.5   1
4.3   1
4.5   1
3.9   0
3.9   0
4.4   1
4.4   1
4.4   1
4.0   1
4.1   1
4.1   1
4.4   1
4.3   1
4.3   1
4.2   1
4.3   1
3.4   0
4.3   1
4.3   1
4.2   1
4.6   1
4.8   1
4.4   1
3.4   0
3.7   0
4.2   1
4.3   1
4.4   1
4.3   1
4.2   1
4.1   1
4.3   1
4.0   1
3.9   0
4.4   1
4.4   1
3.9   0


In [442]:
mobile_Y_train_binarised.shape[0]

341

In [443]:
mobile_X_train_std_values=mobile_X_train_std.values
mobile_X_test_std_values=mobile_X_test_std.values
mobile_Y_train_binarised_values=mobile_Y_train_binarised.values

In [444]:
class MPNeuron:
    def __init__(self):
        self.b=None # it is based on number of parameter in the model
        
    def model(self,x):
        return(sum(x) >= self.b)
    
    def predict(self,X):
        Y=[]
        for x in X:
            result=self.model(x)
            Y.append(result)
        return np.array(Y)
    def fit(self,X,Y):
        accuracy={}
        b=0
        while b <=(X.shape[1]+1):
            self.b=b
            Y_pred=self.predict(X)
            accuracy[b]=accuracy_score(Y_pred,Y)
            print(b,' ',accuracy[b])
            b=b+0.01
            
        best_b=max(accuracy, key = accuracy.get)
        self.b=best_b
        
        print('optimal value:',best_b)
        print('Highest accuracy',accuracy[best_b])

In [445]:
mpneuron=MPNeuron()
mpneuron.fit(mobile_X_train_std_values,mobile_Y_train_binarised_values)

0   0.6979472140762464
0.1   0.6979472140762464
0.2   0.6979472140762464
0.30000000000000004   0.6979472140762464
0.4   0.6979472140762464
0.5   0.6979472140762464
0.6   0.6979472140762464
0.7   0.6979472140762464
0.7999999999999999   0.6979472140762464
0.8999999999999999   0.6979472140762464
0.9999999999999999   0.6979472140762464
1.0999999999999999   0.6979472140762464
1.2   0.6979472140762464
1.3   0.6979472140762464
1.4000000000000001   0.6979472140762464
1.5000000000000002   0.6979472140762464
1.6000000000000003   0.6979472140762464
1.7000000000000004   0.6979472140762464
1.8000000000000005   0.6979472140762464
1.9000000000000006   0.6979472140762464
2.0000000000000004   0.6979472140762464
2.1000000000000005   0.6979472140762464
2.2000000000000006   0.6979472140762464
2.3000000000000007   0.6979472140762464
2.400000000000001   0.6979472140762464
2.500000000000001   0.6979472140762464
2.600000000000001   0.6979472140762464
2.700000000000001   0.6979472140762464
2.800000000000001   

21.600000000000037   0.3020527859237537
21.70000000000004   0.3020527859237537
21.80000000000004   0.3020527859237537
21.90000000000004   0.3020527859237537
22.000000000000043   0.3020527859237537
22.100000000000044   0.3020527859237537
22.200000000000045   0.3020527859237537
22.300000000000047   0.3020527859237537
22.40000000000005   0.3020527859237537
22.50000000000005   0.3020527859237537
22.60000000000005   0.3020527859237537
22.700000000000053   0.3020527859237537
22.800000000000054   0.3020527859237537
22.900000000000055   0.3020527859237537
23.000000000000057   0.3020527859237537
23.10000000000006   0.3020527859237537
23.20000000000006   0.3020527859237537
23.30000000000006   0.3020527859237537
23.400000000000063   0.3020527859237537
23.500000000000064   0.3020527859237537
23.600000000000065   0.3020527859237537
23.700000000000067   0.3020527859237537
23.800000000000068   0.3020527859237537
23.90000000000007   0.3020527859237537
24.00000000000007   0.3020527859237537
24.10000000

43.50000000000035   0.3020527859237537
43.60000000000035   0.3020527859237537
43.70000000000035   0.3020527859237537
43.80000000000035   0.3020527859237537
43.900000000000354   0.3020527859237537
44.000000000000355   0.3020527859237537
44.10000000000036   0.3020527859237537
44.20000000000036   0.3020527859237537
44.30000000000036   0.3020527859237537
44.40000000000036   0.3020527859237537
44.50000000000036   0.3020527859237537
44.600000000000364   0.3020527859237537
44.700000000000365   0.3020527859237537
44.80000000000037   0.3020527859237537
44.90000000000037   0.3020527859237537
45.00000000000037   0.3020527859237537
45.10000000000037   0.3020527859237537
45.20000000000037   0.3020527859237537
45.300000000000374   0.3020527859237537
45.400000000000375   0.3020527859237537
45.50000000000038   0.3020527859237537
45.60000000000038   0.3020527859237537
45.70000000000038   0.3020527859237537
45.80000000000038   0.3020527859237537
45.90000000000038   0.3020527859237537
46.000000000000384 

64.6000000000006   0.3020527859237537
64.7000000000006   0.3020527859237537
64.8000000000006   0.3020527859237537
64.90000000000059   0.3020527859237537
65.00000000000058   0.3020527859237537
65.10000000000058   0.3020527859237537
65.20000000000057   0.3020527859237537
65.30000000000057   0.3020527859237537
65.40000000000056   0.3020527859237537
65.50000000000055   0.3020527859237537
65.60000000000055   0.3020527859237537
65.70000000000054   0.3020527859237537
65.80000000000054   0.3020527859237537
65.90000000000053   0.3020527859237537
66.00000000000053   0.3020527859237537
66.10000000000052   0.3020527859237537
66.20000000000051   0.3020527859237537
66.30000000000051   0.3020527859237537
66.4000000000005   0.3020527859237537
66.5000000000005   0.3020527859237537
66.60000000000049   0.3020527859237537
66.70000000000049   0.3020527859237537
66.80000000000048   0.3020527859237537
66.90000000000047   0.3020527859237537
67.00000000000047   0.3020527859237537
67.10000000000046   0.30205278

85.8999999999994   0.3020527859237537
85.99999999999939   0.3020527859237537
86.09999999999938   0.3020527859237537
86.19999999999938   0.3020527859237537
86.29999999999937   0.3020527859237537
86.39999999999937   0.3020527859237537
86.49999999999936   0.3020527859237537
86.59999999999935   0.3020527859237537
86.69999999999935   0.3020527859237537
86.79999999999934   0.3020527859237537
86.89999999999934   0.3020527859237537
86.99999999999933   0.3020527859237537
optimal value: 8.399999999999986
Highest accuracy 0.718475073313783


In [446]:
Y_test_pred=mpneuron.predict(mobile_X_test_std_values)


In [447]:
mobile_X_test_std_values.shape

(119, 86)

In [448]:
Y_test_pred[0].size

1

In [449]:
test_new['PhoneId'].head()

0     3
1    11
2    13
3    16
4    19
Name: PhoneId, dtype: int64

In [450]:
mobile_X_test_IDs.head()

0     3
1    11
2    13
3    16
4    19
Name: PhoneId, dtype: int64

In [451]:
test_new['PhoneId'].shape

(119,)

In [452]:
Y_test_pred[0].shape

()

In [453]:
submission = pd.DataFrame({'PhoneId':test_new['PhoneId'], 'Class':[1]*Y_test_pred})
submission = submission[['PhoneId', 'Class']]
submission.head()
submission.to_csv("submission.csv", index=False)

In [454]:
print(submission)


     PhoneId  Class
0          3      1
1         11      1
2         13      1
3         16      1
4         19      1
..       ...    ...
114      464      1
115      467      1
116      469      1
117      471      1
118      473      1

[119 rows x 2 columns]
